In [1]:
with open('/kaggle/input/convulational_kan/pytorch/default/1/setup.py') as f:
    print(f.read())


from setuptools import setup, find_packages
from pathlib import Path


def read_file(filename: str) -> str:
    with open(Path(__file__).parent / filename, mode='r', encoding='utf-8') as f:
        return f.read()


PACKAGE_NAME = 'convkan'

long_description = read_file('README.md')
long_description_content_type = 'text/markdown'

python_requires = '>=3.8'
install_requires = read_file('requirements.txt').splitlines()

setup(
    name=PACKAGE_NAME,
    packages=find_packages(),
    version='0.0.1.2',
    author='Vladimir Starostin',
    author_email='vladimir.starostin@uni-tuebingen.de',
    license='MIT',
    description='Convolutional KAN layer',
    long_description=long_description,
    long_description_content_type=long_description_content_type,
    python_requires=python_requires,
    install_requires=install_requires,
)


In [2]:
!pip install --upgrade pip wheel setuptools


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [3]:
!pip install -r /kaggle/input/convulational_kan/pytorch/default/1/requirements.txt


In [4]:
!cp -r /kaggle/input/convulational_kan/pytorch/default/1/convkan .


In [5]:
import torch
from torch import nn
from torchvision import datasets, transforms
from tqdm import tqdm
from torch.utils.data import DataLoader, random_split

from convkan import ConvKAN, LayerNorm2D

# ✅ Define the device (use GPU if available, otherwise use CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
import torch
print(torch.cuda.is_available())


True


In [12]:
# Define transformations (resize, normalize, convert to tensor)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset using ImageFolder
dataset = datasets.ImageFolder(root='/kaggle/input/plantdisease/PlantVillage', transform=transform)

# Split into training and test sets (80% train, 20% test)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
test_loader = DataLoader(test_set, batch_size=8, shuffle=False)

In [13]:
num_classes = len(dataset.classes)

model = nn.Sequential(
    ConvKAN(3, 32, padding=1, kernel_size=3, stride=1),  # 3 channels for RGB
    LayerNorm2D(32),
    ConvKAN(32, 32, padding=1, kernel_size=3, stride=2),
    LayerNorm2D(32),
    ConvKAN(32, num_classes, padding=1, kernel_size=3, stride=2),  # Match num_classes
    nn.AdaptiveAvgPool2d(1),
    nn.Flatten(),
).to(device)


In [14]:
print(dataset.class_to_idx)


{'Pepper__bell___Bacterial_spot': 0, 'Pepper__bell___healthy': 1, 'Potato___Early_blight': 2, 'Potato___Late_blight': 3, 'Potato___healthy': 4, 'Tomato_Bacterial_spot': 5, 'Tomato_Early_blight': 6, 'Tomato_Late_blight': 7, 'Tomato_Leaf_Mold': 8, 'Tomato_Septoria_leaf_spot': 9, 'Tomato_Spider_mites_Two_spotted_spider_mite': 10, 'Tomato__Target_Spot': 11, 'Tomato__Tomato_YellowLeaf__Curl_Virus': 12, 'Tomato__Tomato_mosaic_virus': 13, 'Tomato_healthy': 14}


In [15]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)


In [16]:
epochs = 8  # Set desired number of epochs

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    model.train()
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}')

    for i, (x, y) in enumerate(pbar):
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        
        loss.backward()
        optimizer.step()
        
        pbar.set_description(f'Epoch {epoch+1} | Loss: {loss.item():.4f}')

    # Save model checkpoint each epoch if desired
    torch.save(model.state_dict(), f'plant_disease_model_epoch_{epoch+1}.pth')

print("✅ Training Complete!")
# ✅ Evaluate the model
model.eval()
correct = 0
total = 0

with torch.no_grad():  # No gradient calculation during inference
    pbar = tqdm(test_loader)
    for x, y in pbar:
        # Move test data to the device
        x, y = x.to(device), y.to(device)
        
        y_hat = model(x)                   # Forward pass
        _, predicted = torch.max(y_hat, 1)  # Get predicted class
        total += y.size(0)
        correct += (predicted == y).sum().item()
        
        # Update progress bar with current accuracy
        pbar.set_description(f'Accuracy: {100 * correct / total:.2f}%')

# ✅ Print final test accuracy
print(f'Final Test Accuracy: {100 * correct / total:.2f}%')

Epoch 1/8


Epoch 1 | Loss: 1.7797: 100%|██████████| 2064/2064 [40:19<00:00,  1.17s/it]


Epoch 2/8


Epoch 2 | Loss: 1.5243: 100%|██████████| 2064/2064 [39:32<00:00,  1.15s/it]


Epoch 3/8


...

Epoch 8/8


Epoch 8 | Loss: 1.7827: 100%|██████████| 2064/2064 [40:08<00:00,  1.17s/it]


✅ Training Complete!


Accuracy: 53.71%: 100%|██████████| 516/516 [04:16<00:00,  2.01it/s]

Final Test Accuracy: 53.71%
